In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Dec 22 20:17:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!ls

drive  sample_data


In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.4MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180945 sha256=776a5cb74968eb3ee23ebdcd856beab5b6d734a121727ff6ac64e1f91921ecde
  Stored in directory: /tmp/pip-ephem-wheel-cache-min15de5/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!pip install dgl-cu101

     |████████████████████████████████| 25.0MB 138kB/s 


In [ ]:
!pip install pytorch_pretrained_bert
!pip install emot --upgrade
!pip install transformers==3.5.0

     |████████████████████████████████| 133kB 20.2MB/s 
     |████████████████████████████████| 133kB 31.8MB/s 
     |████████████████████████████████| 7.2MB 41.2MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
ERROR: botocore 1.19.41 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.3MB 14.6MB/s 
     |████████████████████████████████| 2.9MB 37.5MB/s 
     |████████████████████████████████| 890kB 51.8MB/s 
     |████████████████████████████████| 1.1MB 45.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=01dd152e242940954bb7c8f68606c7eae9dff2b426dff10134d3b8d896b632b2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


**Generate Pickle Files for feeding as embedding in Graphs. **

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import dgl
from dgl import DGLGraph
from dgl.data import MiniGCDataset
import dgl.function as fn

from functools import partial

import re
import numpy as np
import pandas as pd

#from transformers import XLMRobertaTokenizer, XLMRobertaModel
#from transformers import BertTokenizer,BertModel

from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel


import spacy
import pickle
import collections

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
import transformers
import os
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
import random

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
BERT_MODEL = 'bert-large-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(BERT_MODEL, never_split = ("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]"))

In [ ]:
#df_train = pd.read_csv('drive/MyDrive/Constraint_Hindi/Constraint_Hindi_Train.csv')
#df_valid = pd.read_csv('drive/MyDrive/Constraint_Hindi/Constraint_Hindi_Valid.csv')

df_train = pd.read_csv('hindi_train_translated.csv')
df_valid = pd.read_csv('hindi_val_translated.csv')
df_test = pd.read_csv('hindi_test_translated.csv')

df_train_hindi = pd.read_csv('Constraint_Hindi_Train.csv')
df_valid_hindi = pd.read_csv('Constraint_Hindi_Valid.csv')
df_test_hindi = pd.read_csv('Constraint_Hindi_Test.csv')

df_train['Unique ID'] = df_train_hindi['Unique ID']
df_valid['Unique ID'] = df_valid_hindi['Unique ID']
df_test['Unique ID'] = df_test_hindi['Unique ID']

In [ ]:
MAX_LEN = 100 #max([len(i.split(" ")) for i in list(df_train['Post'])])

In [ ]:
tokens_lst = []
for _, row in df_train.iterrows():
    
    text = row['Post'].split(' ')
    #print(text)
    text = ' '.join(text[:MAX_LEN])
    #print(text)
    tokens = tokenizer.tokenize(text)
    tokens_lst.append(tokenizer.convert_tokens_to_ids(["[CLS]"] + tokens + ["[SEP]"]))

max_len = max((len(x) for x in tokens_lst))
print(max_len)
tokens = np.zeros((len(tokens_lst), max_len), dtype=np.int64)
for i, row in enumerate(tokens_lst):
    row = np.array(row[:max_len])
    tokens[i, :len(row)] = row

# All sentenses
token_tensor = torch.from_numpy(tokens)

157


In [ ]:
tokens_lst_valid = []
for _, row in df_valid.iterrows():
    
    text = row['Post'].split(' ')
    #print(text)
    text = ' '.join(text[:MAX_LEN])
    #print(text)
    tokens = tokenizer.tokenize(text)
    tokens_lst_valid.append(tokenizer.convert_tokens_to_ids(["[CLS]"] + tokens + ["[SEP]"]))

max_len_valid = max((len(x) for x in tokens_lst_valid))
print(max_len_valid)
tokens = np.zeros((len(tokens_lst_valid), max_len_valid), dtype=np.int64)
for i, row in enumerate(tokens_lst_valid):
    row = np.array(row[:max_len_valid])
    tokens[i, :len(row)] = row

# All sentenses
token_tensor_valid = torch.from_numpy(tokens)

177


In [ ]:
tokens_lst_test = []
for _, row in df_test.iterrows():
    
    text = row['Post'].split(' ')
    #print(text)
    text = ' '.join(text[:MAX_LEN])
    #print(text)
    tokens = tokenizer.tokenize(text)
    tokens_lst_test.append(tokenizer.convert_tokens_to_ids(["[CLS]"] + tokens + ["[SEP]"]))

max_len_valid = max((len(x) for x in tokens_lst_test))
print(max_len_valid)
tokens = np.zeros((len(tokens_lst_test), max_len_valid), dtype=np.int64)
for i, row in enumerate(tokens_lst_test):
    row = np.array(row[:max_len_valid])
    tokens[i, :len(row)] = row

# All sentenses
token_tensor_test = torch.from_numpy(tokens)

176


In [ ]:
#xlm_roberta = XLMRobertaModel.from_pretrained(XLM_ROBERTA_MODEL)
bert_model = transformers.AutoModel.from_pretrained(BERT_MODEL)

In [ ]:
#xlm_roberta.cuda()
bert_model.cuda()
token_tensor = token_tensor.cuda()
token_tensor_valid = token_tensor_valid.cuda()
token_tensor_test = token_tensor_test.cuda()

In [ ]:
import tqdm

bert_outputs = []
with torch.no_grad():
    for i in tqdm.tqdm(range(len(token_tensor))):
        
        bert_output, _ =  bert_model(
                    token_tensor[i].unsqueeze(0), 
                    attention_mask=(token_tensor[i].unsqueeze(0) > 0).long(), 
                    token_type_ids=None, 
                    #output_all_encoded_layers=False) 
        )

        bert_outputs.append(bert_output)

100%|██████████| 5728/5728 [04:05<00:00, 23.34it/s]


In [ ]:

bert_outputs_valid = []
with torch.no_grad():
    for i in tqdm.tqdm(range(len(token_tensor_valid))):
        
        bert_output, _ =  bert_model(
                    token_tensor_valid[i].unsqueeze(0), 
                    attention_mask=(token_tensor_valid[i].unsqueeze(0) > 0).long(), 
                    token_type_ids=None, 
                    #output_all_encoded_layers=False) 
        )

        bert_outputs_valid.append(bert_output)

100%|██████████| 811/811 [00:38<00:00, 20.91it/s]


In [ ]:

bert_outputs_test = []
with torch.no_grad():
    for i in tqdm.tqdm(range(len(token_tensor_test)) , position = 0):
        
        bert_output, _ =  bert_model(
                    token_tensor_test[i].unsqueeze(0), 
                    attention_mask=(token_tensor_test[i].unsqueeze(0) > 0).long(), 
                    token_type_ids=None, 
                    #output_all_encoded_layers=False) 
        )

        bert_outputs_test.append(bert_output)

100%|██████████| 1653/1653 [01:18<00:00, 21.05it/s]


In [ ]:
pickle.dump(tokens_lst, open('token_lst_wto_padding.pkl', "wb"))

In [ ]:
torch.save(bert_outputs , 'bert_outputs.pt')

In [ ]:
pickle.dump(tokens_lst_valid, open('token_lst_wto_padding_valid.pkl', "wb"))

In [ ]:
pickle.dump(bert_outputs_valid, open('bert_outputs_valid.pkl', "wb"))

In [ ]:
pickle.dump(tokens_lst_test, open('token_lst_wto_padding_test.pkl', "wb"))
pickle.dump(bert_outputs_test, open('bert_outputs_test.pkl', "wb"))

**Prepare DGL graphs for the RGCN model**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import dgl
from dgl import DGLGraph
from dgl.data import MiniGCDataset
import dgl.function as fn

from functools import partial

import re
import numpy as np
import pandas as pd

from pytorch_pretrained_bert import BertTokenizer

import spacy
import pickle
import collections

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt

import os
import tqdm
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
from dgl.data.utils import save_graphs,load_graphs
import transformers

Using backend: pytorch


In [ ]:
import random

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
parser = spacy.load('en_core_web_lg')
parser.tokenizer.add_special_case("[UNK]", [{"ORTH": "[UNK]"}])
parser.tokenizer.add_special_case("[SEP]", [{"ORTH": "[SEP]"}])
parser.tokenizer.add_special_case("[PAD]", [{"ORTH": "[PAD]"}])
parser.tokenizer.add_special_case("[CLS]", [{"ORTH": "[CLS]"}])
parser.tokenizer.add_special_case("[MASK]", [{"ORTH": "[MASK]"}])

BERT_MODEL = 'bert-large-uncased'
#BERT_MODEL = 'xlm-roberta-large'
tokenizer = transformers.AutoTokenizer.from_pretrained(BERT_MODEL, never_split = ("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]"))

In [ ]:
token_lst = pickle.load(open('token_lst_wto_padding.pkl', "rb")) # tokens of every sentence without padding
bert_outputs_lst = torch.load('bert_outputs.pt') # list of outputs of bert for every sentence

token_lst_valid = pickle.load(open('token_lst_wto_padding_valid.pkl', "rb")) # tokens of every sentence without padding
bert_outputs_lst_valid = pickle.load(open('bert_outputs_valid.pkl', "rb")) # list of outputs of bert for every sentence

token_lst_test = pickle.load(open('token_lst_wto_padding_test.pkl', "rb")) # tokens of every sentence without padding
bert_outputs_lst_test = pickle.load(open('bert_outputs_test.pkl', "rb")) # list of outputs of bert for every sentence

In [ ]:
bert_outputs_lst[0].shape , bert_outputs_lst_valid[0].shape , bert_outputs_lst_test[0].shape

(torch.Size([1, 157, 1024]),
 torch.Size([1, 177, 1024]),
 torch.Size([1, 176, 1024]))

In [ ]:
def transfer_n_e(nodes, edges):

    num_nodes = len(nodes)
    new_edges = []
    for e1, e2 in edges:
        new_edges.append( [nodes[e1], nodes[e2]] ) 
    return num_nodes, new_edges

all_graphs = []
gcn_offsets = []
for i, sent_token in tqdm.tqdm(enumerate(token_lst),position=0):
    
    sent_token = token_lst[i]
    sent = ' '.join([re.sub("[#]","",token)   for token in tokenizer.convert_ids_to_tokens(sent_token[1:-1])])

    #sent = ' '.join([re.sub("[▁]","",token)   for token in tokenizer.convert_ids_to_tokens(sent_token[1:-1])])
    #print(len(token_lst[i]) , tokenizer.convert_ids_to_tokens(sent_token[1:-1]))
    #print(sent)
    #print(len(sent.split(' ')))

    doc = parser(sent)
    parse_rst = doc.to_json()

    nodes = collections.OrderedDict()
    edges = []
    edge_type = []
    for i_word, word in enumerate(parse_rst['tokens']):
        
        if i_word not in nodes:
            nodes[i_word] = len(nodes) 
            edges.append( [i_word, i_word] )
            edge_type.append(0)
        if word['head'] not in nodes:
            nodes[word['head']] = len(nodes) 
            edges.append( [word['head'], word['head']] )
            edge_type.append(0)

        if word['dep'] != 'ROOT':
                edges.append( [word['head'], word['id']] )
                edge_type.append(1)
                edges.append( [word['id'], word['head']] )
                edge_type.append(2)

    num_nodes, tran_edges = transfer_n_e(nodes, edges)
    
    #print(num_nodes , bert_outputs_lst[i].shape)

    G = dgl.DGLGraph().to('cuda')
    G.add_nodes(num_nodes)
    G.add_edges(list(zip(*tran_edges))[0],list(zip(*tran_edges))[1]) 

    for i_word, word in enumerate(parse_rst['tokens']):
        G.nodes[ [ nodes[i_word] ]].data['h'] = bert_outputs_lst[i][0][i_word + 1].unsqueeze(0).cuda()
        G.nodes[ [ nodes[word['head']] ]].data['h'] = bert_outputs_lst[i][0][word['head'] + 1].unsqueeze(0).cuda()

    edge_norm = []
    for e1, e2 in tran_edges:
        if e1 == e2:
            edge_norm.append(1)
        else:
            edge_norm.append( 1 / (G.in_degree(e2) - 1 ) )


    edge_type = torch.from_numpy(np.array(edge_type)).cuda()
    edge_norm = torch.from_numpy(np.array(edge_norm)).unsqueeze(1).float().cuda()

    G.edata.update({'rel_type': edge_type,})
    G.edata.update({'norm': edge_norm})
    all_graphs.append(G)
    #break

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: DGLGraph.in_degree is deprecated. Please use DGLGraph.in_degrees
  return warnings.warn(message, category=category, stacklevel=1)
5728it [06:07, 15.58it/s]


In [ ]:
all_graphs_valid = []

for i, sent_token in tqdm.tqdm(enumerate(token_lst_valid)):
    
    sent_token = token_lst_valid[i]

    sent = ' '.join([re.sub("[#]","",token)   for token in tokenizer.convert_ids_to_tokens(sent_token[1:-1])])
    #print(i,len(token_lst[i]) , tokenizer.convert_ids_to_tokens(sent_token[1:-1]))
    #print(sent)
    #print(len(sent.split(' ')))

    doc = parser(sent)
    parse_rst = doc.to_json()

    nodes = collections.OrderedDict()
    edges = []
    edge_type = []
    for i_word, word in enumerate(parse_rst['tokens']):
        
        if i_word not in nodes:
            nodes[i_word] = len(nodes) 
            edges.append( [i_word, i_word] )
            edge_type.append(0)
        if word['head'] not in nodes:
            nodes[word['head']] = len(nodes) 
            edges.append( [word['head'], word['head']] )
            edge_type.append(0)

        if word['dep'] != 'ROOT':
                edges.append( [word['head'], word['id']] )
                edge_type.append(1)
                edges.append( [word['id'], word['head']] )
                edge_type.append(2)

    num_nodes, tran_edges = transfer_n_e(nodes, edges)
    
    #print(num_nodes , bert_outputs_lst[i].shape)

    G = dgl.DGLGraph().to('cuda')
    G.add_nodes(num_nodes)
    G.add_edges(list(zip(*tran_edges))[0],list(zip(*tran_edges))[1]) 
    #if i >= 290:
    #  print(i)
    for i_word, word in enumerate(parse_rst['tokens']):
        G.nodes[ [ nodes[i_word] ]].data['h'] = bert_outputs_lst_valid[i][0][i_word + 1].unsqueeze(0).cuda()
        G.nodes[ [ nodes[word['head']] ]].data['h'] = bert_outputs_lst_valid[i][0][word['head'] + 1].unsqueeze(0).cuda()

    edge_norm = []
    for e1, e2 in tran_edges:
        if e1 == e2:
            edge_norm.append(1)
        else:
            edge_norm.append( 1 / (G.in_degree(e2) - 1 ) )


    edge_type = torch.from_numpy(np.array(edge_type)).cuda()
    edge_norm = torch.from_numpy(np.array(edge_norm)).unsqueeze(1).float().cuda()

    G.edata.update({'rel_type': edge_type,})
    G.edata.update({'norm': edge_norm})
    all_graphs_valid.append(G)
    #break

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: DGLGraph.in_degree is deprecated. Please use DGLGraph.in_degrees
  return warnings.warn(message, category=category, stacklevel=1)
811it [00:52, 15.55it/s]


In [ ]:
all_graphs_test = []

for i, sent_token in tqdm.tqdm(enumerate(token_lst_test) , position = 0):
    
    sent_token = token_lst_test[i]

    sent = ' '.join([re.sub("[#]","",token)   for token in tokenizer.convert_ids_to_tokens(sent_token[1:-1])])
    #print(i,len(token_lst[i]) , tokenizer.convert_ids_to_tokens(sent_token[1:-1]))
    #print(sent)
    #print(len(sent.split(' ')))

    doc = parser(sent)
    parse_rst = doc.to_json()

    nodes = collections.OrderedDict()
    edges = []
    edge_type = []
    for i_word, word in enumerate(parse_rst['tokens']):
        
        if i_word not in nodes:
            nodes[i_word] = len(nodes) 
            edges.append( [i_word, i_word] )
            edge_type.append(0)
        if word['head'] not in nodes:
            nodes[word['head']] = len(nodes) 
            edges.append( [word['head'], word['head']] )
            edge_type.append(0)

        if word['dep'] != 'ROOT':
                edges.append( [word['head'], word['id']] )
                edge_type.append(1)
                edges.append( [word['id'], word['head']] )
                edge_type.append(2)

    num_nodes, tran_edges = transfer_n_e(nodes, edges)
    
    #print(num_nodes , bert_outputs_lst[i].shape)

    G = dgl.DGLGraph().to('cuda')
    G.add_nodes(num_nodes)
    G.add_edges(list(zip(*tran_edges))[0],list(zip(*tran_edges))[1]) 
    #if i >= 290:
    #  print(i)
    for i_word, word in enumerate(parse_rst['tokens']):
        G.nodes[ [ nodes[i_word] ]].data['h'] = bert_outputs_lst_test[i][0][i_word + 1].unsqueeze(0).cuda()
        G.nodes[ [ nodes[word['head']] ]].data['h'] = bert_outputs_lst_test[i][0][word['head'] + 1].unsqueeze(0).cuda()

    edge_norm = []
    for e1, e2 in tran_edges:
        if e1 == e2:
            edge_norm.append(1)
        else:
            edge_norm.append( 1 / (G.in_degree(e2) - 1 ) )


    edge_type = torch.from_numpy(np.array(edge_type)).cuda()
    edge_norm = torch.from_numpy(np.array(edge_norm)).unsqueeze(1).float().cuda()

    G.edata.update({'rel_type': edge_type,})
    G.edata.update({'norm': edge_norm})
    all_graphs_test.append(G)
    #break

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: DGLGraph.in_degree is deprecated. Please use DGLGraph.in_degrees
  return warnings.warn(message, category=category, stacklevel=1)
1653it [01:42, 16.16it/s]


In [ ]:
save_graphs("graph_data_valid.bin", all_graphs_valid)
save_graphs("graph_data.bin", all_graphs)
save_graphs("graph_data_test.bin", all_graphs_test)

**Final Model Code**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import dgl
from dgl import DGLGraph
from dgl.data import MiniGCDataset
import dgl.function as fn

from functools import partial

import re
import numpy as np
import pandas as pd

from pytorch_pretrained_bert import BertTokenizer

import spacy
import pickle
import collections

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt

import os
import tqdm
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
from dgl.data.utils import save_graphs,load_graphs
from time import sleep

Using backend: pytorch


In [ ]:
import random

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
token_lst = pickle.load(open('token_lst_wto_padding.pkl', "rb")) # tokens of every sentence without padding
#bert_outputs_lst = torch.load('bert_outputs.pt') # list of outputs of bert for every sentence


token_lst_valid = pickle.load(open('token_lst_wto_padding_valid.pkl', "rb")) # tokens of every sentence without padding
#bert_outputs_lst_valid = pickle.load(open('bert_outputs_valid.pkl', "rb")) # list of outputs of bert for every sentence
 

In [ ]:
all_graphs = load_graphs("graph_data.bin")[0]
all_graphs = [i.to('cuda') for i in all_graphs]
all_graphs_valid = load_graphs("graph_data_valid.bin")[0]
all_graphs_valid = [i.to('cuda') for i in all_graphs_valid]

In [ ]:
RGCN_output_size_1 = 1024
RGCN_output_size_2 = 768
RGCN_output_size_3 = 512
RGCN_output_size_4 = 512

In [ ]:
#df_train = pd.read_csv('drive/MyDrive/Constraint_Hindi/Constraint_Hindi_Train.csv')
#df_valid = pd.read_csv('drive/MyDrive/Constraint_Hindi/Constraint_Hindi_Valid.csv')
df_train = pd.read_csv('hindi_train_translated.csv')
df_valid = pd.read_csv('hindi_val_translated.csv')

In [ ]:
label = {}
c = 0
cats = list(set([j.strip() for i in df_train['Labels Set'] for j in i.split(',')]))
cats.sort()
for j in cats:
  label[j] = c
  c = c+1

In [ ]:
#df_train_hindi = pd.read_csv('Augment_Train_Hindi.csv')
df_train_hindi = pd.read_csv('Constraint_Hindi_Train.csv')
df_valid_hindi = pd.read_csv('Constraint_Hindi_Valid.csv')

In [ ]:
def fix_labels(df):
  final_label = []

  for labels in list(df['Labels Set']):
    labels_tensor = np.zeros(c , dtype=np.int64)
    #print(labels , labels_tensor)
    for i in labels.split(","):
      labels_tensor[label[i.strip()]] = 1
    final_label.append(list(labels_tensor))

  return final_label

In [ ]:
df_train_hindi['Labels'] = fix_labels(df_train_hindi)
df_valid_hindi['Labels'] = fix_labels(df_valid_hindi)

In [ ]:
new_df = df_train_hindi[['Post', 'Labels']].copy()
new_df.columns = ['comment_text' , 'list']
#new_df.head()

new_df_valid = df_valid_hindi[['Post', 'Labels']].copy()
new_df_valid.columns = ['comment_text' , 'list']
#new_df_valid.head()

In [ ]:
import transformers
MAX_LEN = 128
BERT_MODEL = 'bert-base-multilingual-cased'
config = transformers.AutoConfig.from_pretrained(BERT_MODEL)
mBERT_OUTPUT_SIZE = config.hidden_size
tokenizer = transformers.AutoTokenizer.from_pretrained(BERT_MODEL)

In [ ]:
class GPRDataset(Dataset):
    def __init__(self, original_df, graphs , dataframe, tokenizer, max_len , datatype = 'train'):
        
        self.datatype = datatype
        if self.datatype != 'test':
          self.y = original_df['Labels Set']

        self.graphs = graphs
        #self.bert_embeddings = bert_embeddings

        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        #self.targets = self.data.list
        self.max_len = max_len
        
    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):

        if self.datatype != 'test':
          labels = self.y[idx].split(",")
          labels_tensor = np.zeros(c , dtype=np.int64)
          #print(labels , labels_tensor)
          for i in labels:
            #print(i.strip())
            labels_tensor[label[i.strip()]] = 1

        
        comment_text = str(self.comment_text[idx])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        #return self.graphs[idx], self.bert_embeddings[idx] , labels_tensor , torch.tensor(ids, dtype=torch.long).cuda(),torch.tensor(mask, dtype=torch.long).cuda(), torch.tensor(token_type_ids, dtype=torch.long).cuda()
        if self.datatype != 'test':
          return self.graphs[idx], labels_tensor , torch.tensor(ids, dtype=torch.long).cuda(),torch.tensor(mask, dtype=torch.long).cuda(), torch.tensor(token_type_ids, dtype=torch.long).cuda()
        else:
          return self.graphs[idx], torch.tensor(ids, dtype=torch.long).cuda(),torch.tensor(mask, dtype=torch.long).cuda(), torch.tensor(token_type_ids, dtype=torch.long).cuda()

        

In [ ]:
def collate(samples):
    
    #graphs, bert_embeddings , labels , ids , mask , token_type_ids = map(list, zip(*samples))
    graphs , labels , ids , mask , token_type_ids = map(list, zip(*samples))

    batched_graph = dgl.batch(graphs)
    labels = torch.FloatTensor(labels)
    
    #bert_embeddings = [] #torch.stack(bert_embeddings, dim=0).squeeze()

    ids = torch.stack(ids, dim=0).squeeze()
    mask = torch.stack(mask, dim=0).squeeze()
    token_type_ids = torch.stack(token_type_ids, dim=0).squeeze()

    #return batched_graph, bert_embeddings , labels , ids , mask , token_type_ids
    return batched_graph , labels , ids , mask , token_type_ids

In [ ]:
lr_value = 0.0001
total_epoch = 10
def adjust_learning_rate(optimizers, epoch):
    # warm up
    if epoch < 10:
        lr_tmp = 0.00001
    else:
        lr_tmp = lr_value * pow((1 - 1.0 * epoch / 100), 0.9)
    
    if epoch > 36:
        lr_tmp =  0.000015 * pow((1 - 1.0 * epoch / 100), 0.9)
    
    for optimizer in optimizers:
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr_tmp

    return lr_tmp

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score

import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

sigmoid_v = np.vectorize(sigmoid)

In [ ]:
class RGCNLayer(nn.Module):
    def __init__(self, feat_size, num_rels, activation=None, gated = True):
        
        super(RGCNLayer, self).__init__()
        self.feat_size = feat_size
        self.num_rels = num_rels
        self.activation = activation
        self.gated = gated

        self.weight = nn.Parameter(torch.Tensor(self.num_rels, self.feat_size, RGCN_output_size_1))
        # init trainable parameters
        nn.init.xavier_uniform_(self.weight,gain=nn.init.calculate_gain('relu'))
        
        if self.gated:
            self.gate_weight = nn.Parameter(torch.Tensor(self.num_rels, self.feat_size, 1))
            nn.init.xavier_uniform_(self.gate_weight,gain=nn.init.calculate_gain('sigmoid'))
        
    def forward(self, g):
        
        weight = self.weight
        gate_weight = self.gate_weight
        
        def message_func(edges):
            w = weight[edges.data['rel_type']]
            #print(edges.src['h'].shape)
            #print(edges.src['h'])
            msg = torch.bmm(edges.src['h'].unsqueeze(1), w).squeeze()
            msg = msg * edges.data['norm']
            
            if self.gated:
                gate_w = gate_weight[edges.data['rel_type']]
                gate = torch.bmm(edges.src['h'].unsqueeze(1), gate_w).squeeze().reshape(-1,1)
                gate = torch.sigmoid(gate)
                msg = msg * gate
                
            return {'msg': msg}
    
        def apply_func(nodes):
            h = nodes.data['h']
            h = self.activation(h)
            return {'h': h}

        g.update_all(message_func, fn.sum(msg='msg', out='h'), apply_func)

In [ ]:
class RGCNModel(nn.Module):
    def __init__(self, h_dim, num_rels, num_hidden_layers=1, gated = True):
        super(RGCNModel, self).__init__()

        self.h_dim = h_dim
        self.num_rels = num_rels
        self.num_hidden_layers = num_hidden_layers
        self.gated = gated
        
        # create rgcn layers
        self.build_model()
       
    def build_model(self):        
        self.layers = nn.ModuleList() 
        for _ in range(self.num_hidden_layers):
            rgcn_layer = RGCNLayer(self.h_dim, self.num_rels, activation=F.relu, gated = self.gated)
            self.layers.append(rgcn_layer)
    
    def forward(self, g):
        for layer in self.layers:
            layer(g)
        
        rst_hidden = []
        for sub_g in dgl.unbatch(g):
            rst_hidden.append(  sub_g.ndata['h']   )
        return rst_hidden

In [ ]:
class MyHead(nn.Module):
    """The MLP submodule"""
    def __init__(self, gcn_out_size: int, bert_out_size: int):
        super().__init__()
        self.gcn_out_size = gcn_out_size
        
        self.fc = nn.Sequential(
            nn.BatchNorm1d(gcn_out_size + bert_out_size),
            nn.Dropout(0.5),
            nn.Linear(gcn_out_size + bert_out_size, 5),
        )
        for i, module in enumerate(self.fc):
            if isinstance(module, (nn.BatchNorm1d, nn.BatchNorm2d)):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.Linear):
                if getattr(module, "weight_v", None) is not None:
                    nn.init.uniform_(module.weight_g, 0, 1)
                    nn.init.kaiming_normal_(module.weight_v)
                    assert model[i].weight_g is not None
                else:
                    nn.init.kaiming_normal_(module.weight)
                nn.init.constant_(module.bias, 0)
                
    def forward(self, gcn_outputs , bert_embeddings):
        
        gcn_outputs = [ torch.mean(gcn_outputs[i],0).unsqueeze(0) for i in range(len(gcn_outputs)) ]
        
        gcn_extracted_outputs = [gcn_outputs[i].unsqueeze(0).view(gcn_outputs[i].unsqueeze(0).size(0), -1) for i in range(len(gcn_outputs))]
        
        gcn_extracted_outputs = torch.stack(gcn_extracted_outputs, dim=0).squeeze()
        
        embeddings = torch.cat((gcn_extracted_outputs, bert_embeddings), 1) 
        #print(gcn_extracted_outputs.shape)
        
        return self.fc(embeddings)

class GPRModel(nn.Module):
    """The main model."""
    def __init__(self):
        super().__init__()
        self.mBert = transformers.AutoModel.from_pretrained(BERT_MODEL)
        self.RGCN =  RGCNModel(h_dim = 1024, num_rels = 3, num_hidden_layers = 1 , gated = True)
        
        self.head = MyHead(RGCN_output_size_1 , mBERT_OUTPUT_SIZE)
    
    def forward(self, g , ids, mask, token_type_ids):
        gcn_outputs = self.RGCN(g)
        
        _, output_1 = self.mBert(ids, attention_mask = mask, token_type_ids = token_type_ids)

        head_outputs = self.head(gcn_outputs , output_1)
        return head_outputs

In [ ]:
from operator import itemgetter


val_dataset = GPRDataset(original_df = df_valid_hindi, graphs = all_graphs_valid, dataframe=new_df_valid , tokenizer=tokenizer,max_len=MAX_LEN)
train_dataset = GPRDataset(original_df = df_train_hindi, graphs = all_graphs, dataframe=new_df , tokenizer=tokenizer,max_len=MAX_LEN)

train_dataloader = DataLoader(
train_dataset,
collate_fn = collate,
batch_size = 4,
shuffle=True,)

val_dataloader = DataLoader(
val_dataset,
collate_fn = collate,
batch_size = 4,)

model = GPRModel().cuda()
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr_value)
reg_lambda = 0.035

In [ ]:
df_test_hindi = pd.read_csv('Constraint_Hindi_Test.csv')

In [ ]:
df_test_hindi['Labels'] = fix_labels(df_test_hindi)

In [ ]:
new_df_test = df_test_hindi[['Post', 'Labels']].copy()
new_df_test.columns = ['comment_text' , 'list']

In [ ]:
token_lst_test = pickle.load(open('token_lst_wto_padding_test.pkl', "rb")) # tokens of every sentence without padding
bert_outputs_lst_test = pickle.load(open('bert_outputs_test.pkl', "rb")) # list of outputs of bert for every sentence

all_graphs_test = load_graphs("graph_data_test.bin")[0]
all_graphs_test = [i.to('cuda') for i in all_graphs_test]

In [ ]:
test_dataset = GPRDataset(original_df = df_test_hindi, graphs = all_graphs_test, dataframe=new_df_test , tokenizer=tokenizer,max_len=MAX_LEN)

In [ ]:
test_dataloader = DataLoader(
    test_dataset,
    collate_fn = collate,
    batch_size = 3,)

In [ ]:
if os.path.exists('best_model_state_dict.pth'):
  print("loading the best model so far")
  model.load_state_dict(torch.load('best_model_state_dict.pth'))
else:
  print("No Pretrained Model Found !")

No Pretrained Model Found !


In [ ]:


best_val_f1_score = 0

for epoch in tqdm.tqdm(range(total_epoch) , desc = 'Epoch: ' , position=0, leave=True):

  training_loss = 0

  #if epoch % 5 == 0:
  #    print('|',">" * epoch," "*(80-epoch),'|')

  lr = adjust_learning_rate([optimizer],epoch)
  #print("Learning rate = %4f\n" % lr)
  model.train()

  for iter, (batched_graph, labels , ids, mask, token_type_ids) in tqdm.tqdm(enumerate(train_dataloader),position=0, leave=True):

      #bert_embeddings = bert_embeddings.cuda()
      labels = labels.cuda()
      
      prediction = model( batched_graph , ids, mask, token_type_ids)
      
      l2_reg = None
      for w in model.RGCN.parameters():
          if not l2_reg:
              l2_reg = w.norm(2)
          else:
              l2_reg = l2_reg + w.norm(2)  
      for w in model.head.parameters():
          if not l2_reg:
              l2_reg = w.norm(2)
          else:
              l2_reg = l2_reg + w.norm(2)   
      """
      for w in model.BERThead.parameters():
          if not l2_reg:
              l2_reg = w.norm(2)
          else:
              l2_reg = l2_reg + w.norm(2) 
      """
      loss = loss_func(prediction, labels) 
      #print(loss)
        #+ l2_reg * reg_lambda
      #loss = loss_func(prediction, labels)
      training_loss += loss.detach().item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  training_loss = training_loss / (iter+1)
      
  val_loss = 0
  model.eval()
  preds = []
  acts = []
  with torch.no_grad():
      for iter, (batched_graph, labels , ids, mask, token_type_ids) in tqdm.tqdm(enumerate(val_dataloader),position = 0):
          #bert_embeddings = bert_embeddings.cuda()
          labels = labels.cuda()
          prediction = model(batched_graph , ids, mask, token_type_ids)
          loss = loss_func(prediction, labels)
          val_loss += loss.detach().item()
          preds.append(prediction)
          acts.append(labels)
      val_loss = val_loss/(iter + 1)
      
      actual = np.vstack([j.cpu() for i in acts for j in i])
      predi = np.vstack([np.array([1.0 if k > 0.5 else 0.0 for k in sigmoid_v(j.cpu())]) for i in preds for j in i])
      
  val_f1_score =  f1_score(y_true=actual, y_pred=predi, average="weighted")
  #if epoch%20 == 0:
  print('Epoch {}, training_loss {:.4f} , val_loss {:.4f} , f1_score: {:.4f}'.format(epoch, training_loss , val_loss , val_f1_score))
  print(metrics.classification_report(actual,predi))

  test_loss = 0
  model.eval()
  preds = []
  acts = []

  with torch.no_grad():
      for iter, (batched_graph , labels , ids, mask, token_type_ids) in tqdm.tqdm(enumerate(test_dataloader),position=0):
          labels = labels.cuda()
          prediction = model(batched_graph , ids, mask, token_type_ids)
          loss = loss_func(prediction, labels)
          test_loss += loss.detach().item()
          preds.append(prediction)
          acts.append(labels)
      test_loss = test_loss/(iter + 1)
      
      actual = np.vstack([j.cpu() for i in acts for j in i])
      predi = np.vstack([np.array([1.0 if k >= min(0.5,max(sigmoid_v(j.cpu()))) else 0.0 for k in sigmoid_v(j.cpu())]) for i in preds for j in i])
      
  test_f1_score =  f1_score(y_true=actual, y_pred=predi, average="weighted")
  #if epoch%20 == 0:
  print('Epoch {}, training_loss {:.4f} , val_loss {:.4f} , f1_score: {:.4f}'.format(epoch, training_loss , test_loss , test_f1_score))
  print(metrics.classification_report(actual,predi))


  if test_f1_score > best_val_f1_score:
      best_val_f1_score = test_f1_score
      torch.save(model.state_dict(), 'best_model_state_dict.pth') 
      torch.save(model, 'best_model.pt') 
      #if epoch > 36: print('Best val loss found: ', best_val_loss)
      
  torch.cuda.empty_cache()
  sleep(60)

print('This fold, the best f1_score is: ', best_val_f1_score)
#break

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
1432it [05:05,  4.68it/s]
203it [00:14, 14.27it/s]
/usr/local/lib/python3.6/dist-p

Epoch 0, training_loss 0.6627 , val_loss 0.4310 , f1_score: 0.7275
              precision    recall  f1-score   support

           0       0.25      0.78      0.38        77
           1       0.59      0.76      0.66       160
           2       0.36      0.78      0.49       110
           3       0.90      0.96      0.93       435
           4       0.34      0.80      0.48       103

   micro avg       0.55      0.87      0.68       885
   macro avg       0.49      0.81      0.59       885
weighted avg       0.66      0.87      0.73       885
 samples avg       0.68      0.88      0.74       885



551it [00:29, 18.98it/s]


Epoch 0, training_loss 0.6627 , val_loss 0.4336 , f1_score: 0.7272
              precision    recall  f1-score   support

           0       0.23      0.68      0.35       169
           1       0.62      0.76      0.69       334
           2       0.38      0.74      0.50       234
           3       0.91      0.96      0.93       873
           4       0.37      0.82      0.51       219

   micro avg       0.56      0.85      0.68      1829
   macro avg       0.50      0.79      0.59      1829
weighted avg       0.66      0.85      0.73      1829
 samples avg       0.69      0.87      0.74      1829



1432it [05:05,  4.68it/s]
203it [00:14, 14.42it/s]
3it [00:00, 21.78it/s]

...

1432it [05:08,  4.65it/s]
203it [00:14, 14.40it/s]
3it [00:00, 21.89it/s]

Epoch 8, training_loss 0.1746 , val_loss 0.2695 , f1_score: 0.7505
              precision    recall  f1-score   support

           0       0.39      0.19      0.26        77
           1       0.98      0.54      0.69       160
           2       0.42      0.64      0.51       110
           3       0.96      0.97      0.96       435
           4       0.46      0.72      0.56       103

   micro avg       0.75      0.75      0.75       885
   macro avg       0.64      0.61      0.60       885
weighted avg       0.79      0.75      0.75       885
 samples avg       0.75      0.77      0.75       885



551it [00:28, 19.20it/s]


Epoch 8, training_loss 0.1746 , val_loss 0.2742 , f1_score: 0.7587
              precision    recall  f1-score   support

           0       0.49      0.29      0.36       169
           1       0.95      0.56      0.71       334
           2       0.47      0.63      0.54       234
           3       0.95      0.98      0.96       873
           4       0.46      0.71      0.56       219

   micro avg       0.75      0.76      0.76      1829
   macro avg       0.66      0.63      0.63      1829
weighted avg       0.79      0.76      0.76      1829
 samples avg       0.79      0.79      0.78      1829



1144it [04:06,  4.90it/s]

In [ ]:
model.load_state_dict(torch.load('best_model_state_dict.pth'))

In [ ]:
model.eval()
preds = []

with torch.no_grad():
    for iter, (batched_graph , ids, mask, token_type_ids) in tqdm.tqdm(enumerate(test_dataloader),position=0):
        prediction = model(batched_graph , ids, mask, token_type_ids)
        preds.append(prediction)
        
    val_loss = val_loss/(iter + 1)
    
    actual = np.vstack([j.cpu() for i in acts for j in i])
    predi = np.vstack([np.array([1.0 if k >= min(0.5,max(sigmoid_v(j.cpu()))) else 0.0 for k in sigmoid_v(j.cpu())]) for i in preds for j in i])
    

In [ ]:
final_preds = []

for i in predi:
  p = ""
  for index , pred in enumerate(i):
    if pred == 1:
      p = p+cats[index]+","
  final_preds.append(p[:-1])

df_test_hindi['Labels Set'] = final_preds

In [ ]:
df_test_hindi.to_csv('test_preds.csv',index=False)